In [1]:
import pandas as pd
import numpy as np
from typing import List
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document

In [2]:
menu_df = pd.read_csv("Menus/menu1.csv")

In [10]:
documents = [
    Document(
        page_content=f"{row['name']}. {row['description']}. Category: {row['category']}.",
        metadata={
            "id": row["id"],
            "description": row["description"],
            "name": row["name"],
            "price": row["price"],
            "category": row["category"],
            "tags": row["tags"]
        }
    )
    for _, row in menu_df.iterrows()
]

embedding_model_nomic = OllamaEmbeddings(model = 'nomic-embed-text')
menu_vector_store = FAISS.from_documents(documents, embedding_model_nomic)

In [12]:
menu_vector_store.save_local("borcelle_faiss_store")

In [15]:
faiss_vectorstore = FAISS.load_local("borcelle_faiss_store", embedding_model_nomic, allow_dangerous_deserialization=True)

In [ ]:
def semantic_filter(
    query: str,
    vector_store: FAISS,
    embedding_model: OllamaEmbeddings,
    threshold: np.float32 = 0.1,
    k: int = 10
    ) -> List[Document]:

    """
    Perform a semantic search and filter by similarity threshold.
    Returns a list of tuples: (menu item name, similarity score, page content).
    """
    
    # Step 1: Embed query
    query_vector = embedding_model.embed_query(query)

    # Step 2: Search using FAISS
    D, I = vector_store.index.search(np.array([query_vector]), k=k)
    similarities = 1 - D[0]  # Convert distance to similarity

    # Step 3: Filter results above threshold
    results = []
    for i, sim in zip(I[0], similarities):
        if sim >= threshold:
            doc_id = vector_store.index_to_docstore_id[i]
            doc = vector_store.docstore._dict[doc_id]
            results.append(doc)
    
    return results

In [17]:
def search_menu(user_query, category_filter=None, tags_filter=None):

    # Semantic similarity search
    if user_query:
        results = semantic_filter(
            query=user_query,
            vector_store=menu_vector_store,
            embedding_model=embedding_model_nomic,
            threshold=0.2,
            k=15
        )

    else:
    # If no query, return all documents
        results = list(menu_vector_store.docstore._dict.values())


    if category_filter:
        results = [doc for doc in results if doc.metadata["category"].lower() == category_filter.lower()]

    if tags_filter:
        results = [doc for doc in results if tags_filter.lower() in [tag.lower() for tag in doc.metadata["tags"].split(",")]]

    return results

In [19]:
response = search_menu(user_query ="chicken", category_filter=None, tags_filter=None)  # Example usage
for i in [r.metadata['name']+'\t'+str(r.metadata['price'])+'\n'+r.metadata['description']+'\n' for r in response]:  # Print names of the filtered results
    print(i)

Heavenly Herb Chicken	14.99
Roasted chicken with thyme and garlic butter

Chicken Tikka Masala	14.75
Grilled chicken in creamy tomato curry

Chicken Manchurian	12.99
Fried chicken tossed in Indo-Chinese gravy

Kung Pao Chicken	13.99
Spicy stir-fried chicken with peanuts and vegetables

Chicken Fried Rice	11.25
Fried rice with chicken, egg, and vegetables

